In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path
from typing import *

import torch
import torch.optim as optim

import gc
gc.collect()

0

In [ ]:
!pip install pretrainedmodels

     |████████████████████████████████| 58 kB 3.2 MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=6d8f20d2cd6b3af2003a4c94a28fb5713410b064e431c3c2cc9543f343564459
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built pretrainedmodels


In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


In [ ]:
!pip install fastai==1.0.52

     |████████████████████████████████| 219 kB 5.3 MB/s 
     |████████████████████████████████| 78 kB 7.4 MB/s 
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26324 sha256=55407df80de96ba0d99108f513cd62cea9d68ee5831233e8aa65f110fd348f25
  Stored in directory: /root/.cache/pip/wheels/35/f3/15/01aa6571f0a72ee6ae7b827c1491c37a1f72d686fd22b43b0e
Successfully built typing
  Attempting uninstall: fastai
    Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [ ]:
import fastai

In [ ]:
!pip install torch==1.1.0 

     |████████████████████████████████| 676.9 MB 3.9 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.1.0 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.1.0 which is incompatible.


In [ ]:
!pip install torchvision==0.3.0

     |████████████████████████████████| 2.6 MB 5.3 MB/s 
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102


In [ ]:
from fastai import *
from fastai.vision import *
from fastai.text import *

from torchvision.models import *
import pretrainedmodels

#from utils import *
import sys

from fastai.callbacks.tracker import EarlyStoppingCallback
from fastai.callbacks.tracker import SaveModelCallback

In [ ]:
%%bash
pip install pytorch-pretrained-bert

  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from pytorch_pretrained_bert import BertTokenizer
bert_tok = BertTokenizer.from_pretrained(
    "bert-base-uncased",
)

100%|██████████| 231508/231508 [00:00<00:00, 2102519.37B/s]


In [ ]:
class FastAiBertTokenizer(BaseTokenizer):
    """Wrapper around BertTokenizer to be compatible with fast.ai"""
    def __init__(self, tokenizer: BertTokenizer, max_seq_len: int=128, **kwargs):
        self._pretrained_tokenizer = tokenizer
        self.max_seq_len = max_seq_len

    def __call__(self, *args, **kwargs):
        return self

    def tokenizer(self, t:str) -> List[str]:
        """Limits the maximum sequence length"""
        return ["[CLS]"] + self._pretrained_tokenizer.tokenize(t)[:self.max_seq_len - 2] + ["[SEP]"]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Combined_Dataset.csv', index_col=0)

In [ ]:
df.head()

,message,label_maj
0,I got another job! The one at the hospital doi...,0
1,"You have won ?1,000 cash or a ?2,000 prize! To...",1
2,Proj had three reviews and it should be good. ...,0
3,Mm you ask him to come its enough :-),0
4,Yes Bhuwan I have raised with them many times ...,1


In [ ]:
df.dtypes

message      object
label_maj     int64
dtype: object

In [ ]:
df['label_maj'] = df['label_maj'].astype(str)

In [ ]:
df.dtypes

message      object
label_maj    object
dtype: object

In [ ]:
res = pd.get_dummies(df['label_maj'])

In [ ]:
res.dtypes

0    uint8
1    uint8
dtype: object

In [ ]:
res = res.astype(str)

In [ ]:
df = pd.concat([df, res], axis=1, join='inner')

In [ ]:
label_cols = df.label_maj.unique()

In [ ]:
label_cols

array(['0', '1'], dtype=object)

In [ ]:
df.drop('label_maj', axis=1, inplace=True)

In [ ]:
df

,message,0,1
0,I got another job! The one at the hospital doi...,1,0
1,"You have won ?1,000 cash or a ?2,000 prize! To...",0,1
2,Proj had three reviews and it should be good. ...,1,0
3,Mm you ask him to come its enough :-),1,0
4,Yes Bhuwan I have raised with them many times ...,0,1
...,...,...,...
17568,452 even I was present whole day on 22nd Janua...,1,0
17569,I only work from mon to thurs but Sat i cant l...,1,0
17570,🍟Wife Rings Husband.. Husband : I am in office...,0,1
17571,If anyone wants to submit their documents for ...,1,0


In [ ]:
df.dtypes

message    object
0          object
1          object
dtype: object

In [ ]:
train,val = train_test_split(df, shuffle=True, test_size=0.2, random_state=42)

In [ ]:
train.head()

,message,0,1
3108,Yar... I tot u knew dis would happen long ago ...,1,0
5763,Free entry in 2 a weekly comp for a chance to ...,0,1
5082,"Sir, 6 cm was collected in a building.",1,0
16292,How are you babes. Hope your doing ok. I had a...,1,0
15208,"A good friend like you is hard to find, hard t...",0,1


In [ ]:
val.head()

,message,0,1
10791,A new day brings new dreams. Work hard and do ...,0,1
14239,World Cup Football is arriving.....Be careful👇🏼,0,1
9990,She Left Note On ‘Fridge’ = It’s Not ~Working…...,0,1
14745,6th Oct: API (Mini)20th Oct: Wifi (Mini and ...,1,0
9024,See Flood Detection/Related Work/shoaib14.pdf ...,1,0


In [ ]:
fastai_bert_vocab = Vocab(list(bert_tok.vocab.keys()))

In [ ]:
fastai_tokenizer = Tokenizer(tok_func=FastAiBertTokenizer(bert_tok, max_seq_len=256), pre_rules=[], post_rules=[])

In [ ]:
df.dtypes

message    object
0          object
1          object
dtype: object

In [ ]:
lab = label_cols.tolist()

In [ ]:
lab

['0', '1']

In [ ]:
train.head()

,message,0,1
3108,Yar... I tot u knew dis would happen long ago ...,1,0
5763,Free entry in 2 a weekly comp for a chance to ...,0,1
5082,"Sir, 6 cm was collected in a building.",1,0
16292,How are you babes. Hope your doing ok. I had a...,1,0
15208,"A good friend like you is hard to find, hard t...",0,1


In [ ]:
databunch_1 = TextDataBunch.from_df(".", train, val, 
                  tokenizer=fastai_tokenizer,
                  vocab=fastai_bert_vocab,
                  include_bos=False,
                  include_eos=False,
                  text_cols="message",
                  label_cols=lab,
                  bs=32,
                  collate_fn=partial(pad_collate, pad_first=False, pad_idx=0),
             )

/usr/local/lib/python3.7/dist-packages/fastai/core.py:273: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
class BertTokenizeProcessor(TokenizeProcessor):
    def __init__(self, tokenizer):
        super().__init__(tokenizer=tokenizer, include_bos=False, include_eos=False)

class BertNumericalizeProcessor(NumericalizeProcessor):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, vocab=Vocab(list(bert_tok.vocab.keys())), **kwargs)

def get_bert_processor(tokenizer:Tokenizer=None, vocab:Vocab=None):
    """
    Constructing preprocessors for BERT
    We remove sos/eos tokens since we add that ourselves in the tokenizer.
    We also use a custom vocabulary to match the numericalization with the original BERT model.
    """
    return [BertTokenizeProcessor(tokenizer=tokenizer),
            NumericalizeProcessor(vocab=vocab)]

In [ ]:
class BertDataBunch(TextDataBunch):
    @classmethod
    def from_df(cls, path:PathOrStr, train_df:DataFrame, valid_df:DataFrame, test_df:Optional[DataFrame]=None,
                tokenizer:Tokenizer=None, vocab:Vocab=None, classes:Collection[str]=None, text_cols:IntsOrStrs=1,
                label_cols:IntsOrStrs=0, label_delim:str=None, **kwargs) -> DataBunch:
        "Create a `TextDataBunch` from DataFrames."
        p_kwargs, kwargs = split_kwargs_by_func(kwargs, get_bert_processor)
        # use our custom processors while taking tokenizer and vocab as kwargs
        processor = get_bert_processor(tokenizer=tokenizer, vocab=vocab, **p_kwargs)
        if classes is None and is_listy(label_cols) and len(label_cols) > 1: classes = label_cols
        src = ItemLists(path, TextList.from_df(train_df, path, cols=text_cols, processor=processor),
                        TextList.from_df(valid_df, path, cols=text_cols, processor=processor))
        src = src.label_for_lm() if cls==TextLMDataBunch else src.label_from_df(cols=label_cols, classes=classes)
        if test_df is not None: src.add_test(TextList.from_df(test_df, path, cols=text_cols))
        return src.databunch(**kwargs)

In [ ]:
databunch_2 = BertDataBunch.from_df(".", train_df=train, valid_df=val,
                  tokenizer=fastai_tokenizer,
                  vocab=fastai_bert_vocab,
                  text_cols="message",
                  label_cols=lab,
                  bs=8,
                  collate_fn=partial(pad_collate, pad_first=False, pad_idx=0),
             )

/usr/local/lib/python3.7/dist-packages/fastai/core.py:273: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
databunch_2.show_batch()

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


text,target
"[CLS] * dr al ##oys ##ius lo ##h : * * life after 60 * * an interesting article . * life can begin at 60 , it is all in your hands ! many people feel unhappy , health - wise and security - wise , after 60 years of age , owing to the dim ##ini ##shing importance given to them and their opinions . but , it",1
"[CLS] read this please [UNK] young ##ster asked his father : how did you people live before with : no access to technology ##no aero ##planes ##no internet ##no computers ##no dramas ##no tv ##s ##no air con ##s ##no cars ##no mobile phones ? dad replied : "" just like how your generation live today "" : no prayers ##no compassion ##no honor ##no respect ##no character ##no shame",1
[CLS] z ##oma ##to | list of all working promo codes in one post ##bel ##ow are the list of all working promo codes ( maybe specific restaurants / users ) : - 1 . gm ##30 ( pay via pay ##tm and get 30 % cash ##back up ##to a ‚ ¹ ##100 a ##€ “ valid from 7 ##am to 11 ##am ) a ##€ “ valid till,1
[CLS] quotes on gandhi jaya ##nti : 1 . an eye for eye only ends up making the whole world blind . ( ब ##द ##ल क ##ी भ ##ा ##व ##न ##ा व ##ि ##श ##व क ##ो अ ##ध ##ा ब ##न ##ा स ##क ##त ##ी ह ) 2 . you must be the change you wish to see in the world . ( व ##ह प,1
"[CLS] highlights of budget 2019 * tax * 1 . within 2 years , tax assessment will be done electronically ##2 . it returns processing in just 24 hours ##3 . minimum 14 % revenue of gs ##t to states by central govt . 4 . custom duty has abolished from 36 capital goods ##5 . recommendations to gs ##t council for reducing gs ##t rates for home buyers ##6",1


In [ ]:
from pytorch_pretrained_bert.modeling import BertConfig, BertForSequenceClassification, BertForNextSentencePrediction, BertForMaskedLM
bert_model_class = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

100%|██████████| 407873900/407873900 [00:11<00:00, 34842144.41B/s]


In [ ]:
loss_func = nn.BCEWithLogitsLoss()

In [ ]:
acc_02 = partial(accuracy_thresh, thresh=0.8)

In [ ]:
model = bert_model_class

In [ ]:
from fastai.callbacks import *

learner = Learner(
    databunch_2, model,
    loss_func=loss_func, model_dir='/temp/model', metrics=acc_02,
)

In [ ]:
def bert_clas_split(self) -> List[nn.Module]:
    
    bert = model.bert
    embedder = bert.embeddings
    pooler = bert.pooler
    encoder = bert.encoder
    classifier = [model.dropout, model.classifier]
    n = len(encoder.layer)//3
    print(n)
    groups = [[embedder], list(encoder.layer[:n]), list(encoder.layer[n+1:2*n]), list(encoder.layer[(2*n)+1:]), [pooler], classifier]
    return groups

In [ ]:
x = bert_clas_split(model)

4


In [ ]:
learner.split([x[0], x[1], x[2], x[3], x[5]])

Learner(data=TextClasDataBunch;

Train: LabelList (14058 items)
x: TextList
[CLS] ya ##r . . . i to ##t u knew di ##s would happen long ago already . [SEP],[CLS] free entry in 2 a weekly com ##p for a chance to win an ipod . tx ##t pod to 80 ##18 ##2 to get entry ( st ##d tx ##t rate ) t & c ' s apply 08 ##45 ##28 ##100 ##7 ##3 for details 18 + [SEP],[CLS] sir , 6 cm was collected in a building . [SEP],[CLS] how are you babe ##s . hope your doing ok . i had a shit nights sleep . i fell asleep at 5 . ia ##om kn ##acker ##ed and ia ##om dread ##ing work tonight . what are thou up ##to tonight . x [SEP],[CLS] a good friend like you is hard to find , hard to lose and trust me you are unable to forget . 14 . true friendship is seen through the heart , not through the eyes . 15 . a friend is one who knows all about you and loves you just the same . 16 . friends listen to what you say . best friends listen to what you don ' t say . [UNK] [SEP]
y: MultiCategoryList
0,1,0,0,1
Path: .;

Valid: L

In [ ]:
#learner.lr_find()

epoch,train_loss,valid_loss,accuracy_thresh,time


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


RuntimeError: ignored

In [ ]:
#learner.recorder.plot()

AttributeError: ignored

In [ ]:
learner.fit_one_cycle(2, max_lr=slice(1e-5, 5e-4), moms=(0.8,0.7), pct_start=0.2, wd =(1e-7, 1e-5, 1e-4, 1e-3, 1e-2))

epoch,train_loss,valid_loss,accuracy_thresh,time
0,0.204590,0.115507,0.956614,08:11
1,0.073517,0.068338,0.977525,08:17


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
learner.save('/content/drive/MyDrive/im-text-best-model.pkl')

In [ ]:
learner.load('/content/drive/MyDrive/im-text-best-model.pkl')
#learner.save_encoder('/content/drive/MyDrive/ml/best-model')

Learner(data=TextClasDataBunch;

Train: LabelList (14058 items)
x: TextList
[CLS] ya ##r . . . i to ##t u knew di ##s would happen long ago already . [SEP],[CLS] free entry in 2 a weekly com ##p for a chance to win an ipod . tx ##t pod to 80 ##18 ##2 to get entry ( st ##d tx ##t rate ) t & c ' s apply 08 ##45 ##28 ##100 ##7 ##3 for details 18 + [SEP],[CLS] sir , 6 cm was collected in a building . [SEP],[CLS] how are you babe ##s . hope your doing ok . i had a shit nights sleep . i fell asleep at 5 . ia ##om kn ##acker ##ed and ia ##om dread ##ing work tonight . what are thou up ##to tonight . x [SEP],[CLS] a good friend like you is hard to find , hard to lose and trust me you are unable to forget . 14 . true friendship is seen through the heart , not through the eyes . 15 . a friend is one who knows all about you and loves you just the same . 16 . friends listen to what you say . best friends listen to what you don ' t say . [UNK] [SEP]
y: MultiCategoryList
0,1,0,0,1
Path: .;

Valid: L

In [ ]:
text1_dis = "You have won ?1,000 cash or a ?2,000 prize!"
text2_dis = "World Cup Football is arriving.....Be careful"
text1_not_dis = "Hey I need some help with the assignment."
text2_not_dis = "The paper review is next week."

In [ ]:
learner.predict(text1_dis)

(MultiCategory 1, tensor([0., 1.]), tensor([0.0027, 0.9973]))

In [ ]:
learner.predict(text2_dis)

(MultiCategory 1, tensor([0., 1.]), tensor([0.1220, 0.8774]))

In [ ]:
learner.predict(text1_not_dis)

(MultiCategory 0, tensor([1., 0.]), tensor([0.9971, 0.0029]))

In [ ]:
learner.predict(text2_not_dis)

(MultiCategory 0, tensor([1., 0.]), tensor([0.9971, 0.0028]))

In [ ]:
text = "Good morning"
learner.predict(text)

(MultiCategory 0, tensor([1., 0.]), tensor([0.8420, 0.1598]))

In [ ]:
np.argmax((learner.predict(text))[1])

tensor(1)

In [ ]:
import csv

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/ml/test.csv',escapechar = "\\", quoting = csv.QUOTE_NONE)

In [ ]:
df.BROWSE_NODE_ID.value_counts()

In [ ]:
test_df.head()

In [ ]:
test_df['PRODUCT_ID']

In [ ]:
test_df['BROWSER_NODE_ID'] = ''

In [ ]:
test_df.drop('DESCRIPTION', axis=1, inplace=True)

In [ ]:
test_df.drop('BULLET_POINTS', axis=1, inplace=True)
test_df.drop('BRAND', axis=1, inplace=True)

In [ ]:
pred=[]

In [ ]:
for index, row in df.iterrows():
    pred.append(np.argmax((learner.predict(text))[2]))

In [ ]:
prd = pd.DataFrame(pred)

In [ ]:
prd['0'].unique()

In [ ]:
prd.head(-50)

In [ ]:
submission = pd.merge(prd, test_df['PRODUCT_ID'], )

In [ ]:
submission = pd.concat([prd, test_df['PRODUCT_ID']], axis=1, join='inner')

In [ ]:
submission

In [ ]:
submission.to_csv